# Processing CSV data using LLMs on Amazon Bedrock

[Semi-structured data](https://en.wikipedia.org/wiki/Semi-structured_data) is a form of structured data that does not obey the tabular structure of data models associated with relational databases or other forms of data tables, but nonetheless contains tags or other markers to separate semantic elements and enforce hierarchies of records and fields within the data.

In complex [Generative AI](https://en.wikipedia.org/wiki/Generative_artificial_intelligence) use cases that involve [Large Language Models (LLMs)](https://en.wikipedia.org/wiki/Large_language_model), we often come across the need to process semi-structured data through LLMs.

This notebook will walk you through examples of processing [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) data through natural language queries by using the LLMs hosted on [Amazon Bedrock](https://aws.amazon.com/bedrock/). These would be,
* Data extraction with conditions
* Filtering
* Aggregation
* Sorting
* Transformations

We will use [LangChain](https://www.langchain.com/) to simplify the process of constructing the prompts and interacting with the LLMs. In the process of working through this notebook, you will learn how to setup the Amazon Bedrock client environment, configure security permissions and use prompt templates in LangChain.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> LLMs are not a good fit for some of these operations as you will notice in the prompt responses further down in the notebook. For those, you will be better off performing those operations outside the LLMs and passing the results to the LLMs for further processing. One way to achieve this is through <a href="https://python.langchain.com/docs/modules/model_io/chat/function_calling">function calling</a> but that is out of scope for this notebook.
</div>

<div class="alert alert-block alert-info">
<b>Note:</b>
    <ul>
        <li>This notebook should only be run from within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html">Amazon SageMaker Notebook instance</a> or within an <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated.html">Amazon SageMaker Studio Notebook</a>.</li>
        <li>This notebook uses text based models along with their versions that were available at the time of writing. Update these as required.</li>
        <li>At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions. Follow the guidance in the <i>Organize imports</i> section of this notebook.</li>
        <li>This notebook is recommended to be run with a minimum instance size of <i>ml.m5.xlarge</i> and
            <ul>
                <li>With <i>Amazon Linux 2, Jupyter Lab 3</i> as the platform identifier on an Amazon SageMaker Notebook instance.</li>
                <li> (or)
                <li>With <i>Data Science 3.0</i> as the image on an Amazon SageMaker Studio Notebook.</li>
            <ul>
        </li>
        <li>At the time of this writing, the most relevant latest version of the Kernel for running this notebook,
            <ul>
                <li>On an Amazon SageMaker Notebook instance was <i>conda_python3</i></li>
                <li>On an Amazon SageMaker Studio Notebook was <i>Python 3</i></li>
            </ul>
        </li>
    </ul>
</div>

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)

    1. [Check and configure access to the Internet](#Check%20and%20configure%20access%20to%20the%20Internet)

    2. [Install required software libraries](#Install%20required%20software%20libraries)
    
    3. [Configure logging](#Configure%20logging)
        
        1. [System logs](#Configure%20system%20logs)
        
        2. [Application logs](#Configure%20application%20logs)
    
    4. [Organize imports](#Organize%20imports)
    
    5. [Create common objects](#Create%20common%20objects)
    
    6. [Enable model access in Amazon Bedrock](#Enable%20model%20access%20in%20Amazon%20Bedrock)
    
    7. [Check and configure security permissions](#Check%20and%20configure%20security%20permissions)
    
    8. [List the available models](#List%20the%20available%20models)

 2. [Prompt examples](#Prompt%20examples)
 
    1. [Prompt 1](#Prompt%201)
     
        1. [AI21 Labs Jurassic](#AI21%20Labs%20Jurassic%20prompt%201)
        
        2. [Anthropic Claude](#Anthropic%20Claude%20prompt%201)
        
        3. [Cohere Command](#Cohere%20Command%20prompt%201)
        
        4. [Mistral](#Mistral%20prompt%201)
        
        5. [Meta LLAMA 2](#Meta%20LLAMA%202%20prompt%201)
        
        6. [Amazon Titan](#Amazon%20Titan%20prompt%201)
 
 3. [Cleanup](#Cleanup)
 
 4. [Conclusion](#Conclusion)
 
 5. [Frequently Asked Questions (FAQs)](#FAQs)

##  1. Complete prerequisites <a id ='Complete%20prerequisites'> </a>

Check and complete the prerequisites.

###  A. Check and configure access to the Internet <a id ='Check%20and%20configure%20access%20to%20the%20Internet'> </a>
This notebook requires outbound access to the Internet to download the required software updates and to download the dataset.  You can either provide direct Internet access (default) or provide Internet access through an [Amazon VPC](https://aws.amazon.com/vpc/).  For more information on this, refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/appendix-notebook-and-internet-access.html).

### B. Install required software libraries <a id ='Install%20required%20software%20libraries'> </a>
This notebook requires the following libraries:
* [SageMaker Python SDK version 2.x](https://sagemaker.readthedocs.io/en/stable/v2.html)
* [Python 3.10.x](https://www.python.org/downloads/release/python-3100/)
* [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
* [LangChain](https://www.langchain.com/)

Run the following cell to install the required libraries.

<div class="alert alert-block alert-warning">  
    <b>Note:</b> At the end of the installation, the Kernel will be forcefully restarted immediately. Please wait 10 seconds for the kernel to come back before running the next cell.
</div>

In [ ]:
!pip install boto3==1.34.62
!pip install langchain==0.1.12
!pip install sagemaker==2.212.0

import IPython

IPython.Application.instance().kernel.do_shutdown(True)

### C. Configure logging <a id ='Configure%20logging'> </a>

####  a. System logs <a id='Configure%20system%20logs'></a>

System logs refers to the logs generated by the notebook's interactions with the underlying notebook instance. Some examples of these are the logs generated when loading or saving the notebook.

These logs are automatically setup when the notebook instance is launched.

These logs can be accessed through the [Amazon CloudWatch Logs](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/WhatIsCloudWatchLogs.html) console in the same AWS Region where this notebook is running.
* When running this notebook in an Amazon SageMaker Notebook instance, navigate to the following location,
    * <i>CloudWatch > Log groups > /aws/sagemaker/NotebookInstances > {notebook-instance-name}/jupyter.log</i>
* When running this notebook in an Amazon SageMaker Studio Notebook, navigate to the following locations,
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/KernelGateway/{notebook-instance-name}</i>
    * <i>CloudWatch > Log groups > /aws/sagemaker/studio > {sagmaker-domain-name}/{user-name}/JupyterServer/default</i>

Run the following cell to print the name of the underlying notebook instance.

In [ ]:
import json

notebook_name = ''
resource_metadata_path = '/opt/ml/metadata/resource-metadata.json'
with open(resource_metadata_path, 'r') as metadata:
    notebook_name = (json.load(metadata))['ResourceName']
print("Notebook instance name: '{}'".format(notebook_name))

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  D. Organize imports <a id ='Organize%20imports'> </a>

Organize all the library and module imports for later use.

In [ ]:
import boto3
import langchain
import sagemaker
import sys
from botocore.config import Config

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

Print the installed versions of some of the important libraries.

In [ ]:
logging.info("Python version : {}".format(sys.version))
logging.info("Boto3 version : {}".format(boto3.__version__))
logging.info("SageMaker Python SDK version : {}".format(sagemaker.__version__))
logging.info("LangChain version : {}".format(langchain.__version__))

###  E. Create common objects <a id='Create%20common%20objects'> </a>

Get the current AWS Region (where this notebook is running) and the SageMaker Session. These will be used to initialize some of the clients to AWS services using the boto3 APIs.

<div class="alert alert-block alert-warning">  
<b>Note:</b> At the time of writing this notebook, Amazon Bedrock was only available in <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-regions.html">these supported AWS Regions</a>. If you are running this notebook from any other AWS Region, then you have to change the Amazon Bedrock client's region and/or endpoint URL parameters to one of those supported AWS Regions. In order to do this, this notebook will use the value specified in the environment variable named <mark>AMAZON_BEDROCK_REGION</mark>. If this is not specified, then the notebook will default to <mark>us-west-2 (Oregon)</mark> for Amazon Bedrock.
</div>



In [ ]:
# Get the AWS Region, SageMaker Session and IAM Role references
my_session = boto3.session.Session()
logging.info("SageMaker Session: {}".format(my_session))
my_iam_role = sagemaker.get_execution_role()
logging.info("Notebook IAM Role: {}".format(my_iam_role))
my_region = my_session.region_name
logging.info("Current AWS Region: {}".format(my_region))

# Explicity set the AWS Region for Amazon Bedrock clients
AMAZON_BEDROCK_DEFAULT_REGION = "us-west-2"
br_region = os.environ.get('AMAZON_BEDROCK_REGION')
if br_region is None:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
elif len(br_region) == 0:
    br_region = AMAZON_BEDROCK_DEFAULT_REGION
logging.info("AWS Region for Amazon Bedrock: {}".format(br_region))

Set the timeout and retry configurations that will be applied to all the boto3 clients used in this notebook.

In [ ]:
# Increase the standard time out limits in the boto3 client from 1 minute to 3 minutes
# and set the retry limits
my_boto3_config = Config(
    connect_timeout = (60 * 3),
    read_timeout = (60 * 3),
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

Create the rest of the common objects.

In [ ]:
# Create the Amazon Bedrock client
bedrock_client = boto3.client("bedrock", region_name = br_region, endpoint_url = "https://bedrock.{}.amazonaws.com"
                              .format(br_region), config = my_boto3_config)

# Create the Amazon Bedrock runtime client
bedrock_rt_client = boto3.client("bedrock-runtime", region_name = br_region, config = my_boto3_config)

# Specify the path to the directories that will contain the prompt
# templates and data for the prompts
prompt_1_templates_dir = os.path.join(os.getcwd(), "prompt_templates", "csv", "prompt1")
prompt_2_templates_dir = os.path.join(os.getcwd(), "prompt_templates", "csv", "prompt2")
data_dir = os.path.join(os.getcwd(), "data", "csv")

###  F. Enable model access in Amazon Bedrock <a id ='Enable%20model%20access%20in%20Amazon%20Bedrock'> </a>

<div class="alert alert-block alert-danger">
    <b>Note:</b> Before invoking any model in Amazon Bedrock, enable access to that model by following the instructions <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html">here</a>. In addition, for Anthropic models, you need to submit the use case details. Otherwise, you will get an authorization error.
</div>

Run the following cell to print the Amazon Bedrock model access page URL for the AWS Region that was selected earlier.

In [ ]:
# Print the Amazon Bedrock model access page URL
logging.info("Amazon Bedrock model access page - https://{}.console.aws.amazon.com/bedrock/home?region={}#/modelaccess"
             .format(br_region, br_region))

<div class="alert alert-block alert-warning">  
<b>Note:</b> You will have to do this manually after reading the End User License Agreement (EULA) for each of the models that you want to enable. Unless you explicitly disable it, this is a one-time setup for each model in an AWS account.
</div>

###  G. Check and configure security permissions <a id ='Check%20and%20configure%20security%20permissions'> </a>
This notebook uses the IAM role attached to the underlying notebook instance.  To view the name of this role, run the following cell.

This IAM role should have the following permissions,

1. Full access to invoke Large Language Models (LLMs) on Amazon Bedrock.
2. Access to write to Amazon CloudWatch Logs.

Run the following cell to print the details of the IAM role attached to the underlying notebook instance.

In [ ]:
# Print the IAM role ARN and console URL
logging.info("This notebook's IAM role is '{}'".format(my_iam_role))
arn_parts = my_iam_role.split('/')
logging.info("Details of this IAM role are available at https://{}.console.aws.amazon.com/iamv2/home?region={}#/roles/details/{}?section=permissions"
             .format(my_region, my_region, arn_parts[len(arn_parts) - 1]))

###  H. List the available models <a id='List%20the%20available%20models'> </a>

Running the following cell will list all available LLMs on Amazon Bedrock that have 'TEXT' as at least one of the input and output modalities. The results be will filtered further to show only those LLMs that are offered through the On-Demand throughput pricing model. This will help you pick the model-ids that you will use further down in this notebook.

For more information on this, refer [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns).

In [ ]:
# List all the available text based LLMs in Amazon Bedrock with On-Demand throughput pricing
models_info = ''
response = bedrock_client.list_foundation_models(byOutputModality = "TEXT", byInferenceType = "ON_DEMAND")
model_summaries = response["modelSummaries"]
models_info = models_info + "\n"
models_info = models_info + "-".ljust(125, "-") + "\n"
models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format("Provider Name", "Model Name", "Input Modalities",
                                                          "Output Modalities", "Model Id")
models_info = models_info + "-".ljust(125, "-")
for model_summary in model_summaries:
    # Check for 'TEXT' modality in both input and output (./scripts/helper_functions.py) and process
    if does_modality_exists(model_summary["inputModalities"],
                            model_summary["outputModalities"], 'TEXT'):
        models_info = models_info + "\n"
        models_info = models_info + "{:<15} {:<30} {:<20} {:<20} {:<40}".format(model_summary["providerName"],
                                                                                model_summary["modelName"],
                                                                                "|".join(model_summary["inputModalities"]),
                                                                                "|".join(model_summary["outputModalities"]),
                                                                                model_summary["modelId"])
models_info = models_info + "-".ljust(125, "-") + "\n"
logging.info("Displaying available models in the '{}' Region:".format(br_region) + models_info)

## 2. Prompt examples <a id ='Prompt%20examples'> </a>

In this section, we will take some sample CSV files and construct prompts that will ask questions or provide instructions on what we want to lookup from them. We will also show examples of how to get the LLMs to perform data extraction, sorting, aggregation and filtering operations on the CSV data.

In our examples, we want the LLMs to generate the most probable response. So we will set the `temperature` to `0`.

<div class="alert alert-block alert-info">
    <b>Note:</b> Constructing the optimal prompt for your requirement is both a science and an art. It involes experimentation and iteration. To start with, always refer to the model provider's documentation to get an idea of the constructs and best practices for prompting their models. Here are the links:
    <ul>
        <li>AI21 Labs Jurassic models, refer <a href="https://docs.ai21.com/docs/prompt-engineering">here</a>.</li>
        <li>Anthropic Claude models, refer <a href="https://docs.anthropic.com/claude/docs/constructing-a-prompt">here</a>.</li>
        <li>Cohere Command models, refer <a href="https://txt.cohere.com/constructing-prompts/">here</a>.</li>
        <li>Mistral models, refer <a href="https://docs.mistral.ai/guides/prompting-capabilities/">here</a>.</li>
        <li>Meta LLAMA 2 models, refer <a href="https://llama.meta.com/get-started#prompting">here</a>.</li>
        <li>Amazon Titan models, refer <a href="https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf">here</a>.</li>
    </ul>
</div>

### A. Prompt 1 <a id ='Prompt%201'> </a>

In this prompt,

* We will take a CSV file named `books.csv` that contains data about some books.
* This will be a [zero-shot prompt](https://www.promptingguide.ai/techniques/zeroshot).
* The call-to-action i.e. the ask from the LLM,
    * Will be a single turn conversation and not a back-and-forth conversation.
    * Will be questions from that CSV file. These questions will be in natural language and the responses from the LLM will either be natural language or a CSV based on the question. You will see that the LLM will need to parse the CSV, loop through items, perform some math calculations and read the descriptions in order to respond. 
    * Will be instructions for the LLM to convert that CSV to other data formats.  

Note that some books are sequels to the other books listed in that CSV. Where applicable, the LLMs should pay attention to this for an effective response. For example, observe the response to the call-to-action `"I am interested in reading about the aftermath of the fall of nanotechnology. What should I read?"`

To get started, view the `books.csv` file by running the following cell.

In [ ]:
!cat data/csv/books.csv

Now set the inference parameters and the call-to-action.

Uncomment the various `call_to_action` lines in the below cell one at a time and try out each model.

In [ ]:
# Specify the inference parameters
temperature = 0
max_response_token_length = -1

# Read the data (./scripts/helper_functions.py) from the file
# that contains the CSV data to be used in this prompt
csv_data = read_file(data_dir, 'books.csv')

#### Data extraction with conditions (NOTE: You will see accurate results almost all the time)
#call_to_action = 'get me the book snippet where Cynthia Randall is the author.'
call_to_action = 'what is the newest publication?'
#call_to_action = 'what is the the cheapest book?'
#call_to_action = 'what is the oldest title?'
#call_to_action = 'what is the price of Jungle Book?'

#### Filtering (NOTE: You will see accurate results almost all the time)
#call_to_action = 'get me the snippets of all computer related books.'
#call_to_action = 'I am interested in reading about the aftermath of the fall of nanotechnology. What should I read?'
#call_to_action = 'what are the titles of the cheapest books?'

#### Aggregation (NOTE: Except for the total cost call-to-action, you will see accurate results for others almost all the time)
#call_to_action = 'how many books are there in the catalog?'
#call_to_action = 'get all titles that belong to the fantasy genre.'
#call_to_action = 'which author has the most titles?'
#call_to_action = 'I want to buy all the books. How much will it cost?'

#### Sorting (NOTE: You will NOT see accurate results most of the time)
#call_to_action = 'consider every line as a record. Then, sort the records in ascending order of publication date.'
#call_to_action = 'consider every line as a record. Then, sort the records in descending order of price.'

#### Transformations (NOTE: You will see accurate results almost all the time)
#call_to_action = 'convert this CSV to a HTML table.'
#call_to_action = 'convert this CSV to XML.'
#call_to_action = 'convert this CSV to JSON.'

####  A. AI21 Labs Jurassic <a id ='AI21%20Labs%20Jurassic%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/AI21_Labs_Jurassic_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-id
#model_id = "ai21.j2-mid-v1"
model_id = "ai21.j2-ultra-v1"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'AI21_Labs_Jurassic_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

####  B. Anthropic Claude <a id ='Anthropic%20Claude%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/Anthropic_Claude_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-id
#model_id = "anthropic.claude-instant-v1"
#model_id = "anthropic.claude-v2"
#model_id = "anthropic.claude-v2:1"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
#model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'Anthropic_Claude_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

####  C. Cohere Command <a id ='Cohere%20Command%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/Cohere_Command_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-ids along with their inference parameters
#model_id = "cohere.command-light-text-v14"
model_id = "cohere.command-text-v14"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'Cohere_Command_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

####  D. Mistral <a id ='Mistral%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/Mistral_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-ids along with their inference parameters
#model_id = "mistral.mistral-7b-instruct-v0:2"
model_id = "mistral.mixtral-8x7b-instruct-v0:1"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'Mistral_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

####  E. Meta LLAMA 2 <a id ='Meta%20LLAMA%202%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/Meta_LLAMA_2_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-ids along with their inference parameters
#model_id = "meta.llama2-13b-chat-v1"
model_id = "meta.llama2-70b-chat-v1"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'Meta_LLAMA_2_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

####  F. Amazon Titan <a id ='Amazon%20Titan%20prompt%201'> </a>

View the prompt template by running the following cell.

In [ ]:
!cat prompt_templates/csv/prompt1/Amazon_Titan_prompt1_for_CSV.txt

Now select the model-id, run the following cell and observe the output.

In [ ]:
# Specify the model-ids along with their inference parameters
#model_id = "amazon.titan-text-lite-v1"
model_id = "amazon.titan-text-express-v1"

# Prepare the prompt and invoke the LLM (./scripts/helper_functions.py)
single_turn_conversation = process_prompt_1(model_id, bedrock_rt_client, temperature, max_response_token_length,
                                            prompt_1_templates_dir, 'Amazon_Titan_prompt1_for_CSV.txt',
                                            csv_data, call_to_action)

## 3. Cleanup <a id='Cleanup'></a>

As a best practice, you should delete AWS resources that are no longer required.  This will help you avoid incurring unncessary costs.

The minimum cleanup required for this notebook would be shutdown the instance on which this notebook is running.

## 4. Conclusion <a id='Conclusion'></a>

We have now seen how to use natural language queries to process CSV data using the LLMs hosted on [Amazon Bedrock](https://aws.amazon.com/bedrock/). Through this, you were able to learn how to configure model specific prompts, use prompt templates and observe the LLM behaviors for various prompts and call-to-action scenarios.

## 5. Frequently Asked Questions (FAQs) <a id='FAQs'></a>

**Q: What AWS services are used in this notebook?**

Amazon Bedrock, AWS Identity and Access Management (IAM), Amazon CloudWatch, and Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook depending on what you use to run the notebook.

**Q: Will Amazon Bedrock capture and store my data?**

Amazon Bedrock doesn't use your prompts and continuations to train any AWS models or distribute them to third parties. Your training data isn't used to train the base Amazon Titan models or distributed to third parties. Other usage data, such as usage timestamps, logged account IDs, and other information logged by the service, is also not used to train the models.

Amazon Bedrock uses the fine tuning data you provide only for fine tuning an Amazon Titan model. Amazon Bedrock doesn't use fine tuning data for any other purpose, such as training base foundation models.

Each model provider has an escrow account that they upload their models to. The Amazon Bedrock inference account has permissions to call these models, but the escrow accounts themselves don't have outbound permissions to Amazon Bedrock accounts. Additionally, model providers don't have access to Amazon Bedrock logs or access to customer prompts and continuations.

Amazon Bedrock doesn’t store or log your data in its service logs.

**Q: What models are supported by Amazon Bedrock?**

Go [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html).

**Q: What is the difference between On-demand and Provisioned Throughput in Amazon Bedrock?**

With the On-Demand mode, you only pay for what you use, with no time-based term commitments. For text generation models, you are charged for every input token processed and every output token generated. For embeddings models, you are charged for every input token processed. A token is comprised of a few characters and refers to the basic unit that a model learns to understand user input and prompt to generate results. For image generation models, you are charged for every image generated.

With the Provisioned Throughput mode, you can purchase model units for a specific base or custom model. The Provisioned Throughput mode is primarily designed for large consistent inference workloads that need guaranteed throughput. Custom models can only be accessed using Provisioned Throughput. A model unit provides a certain throughput, which is measured by the maximum number of input or output tokens processed per minute. With this Provisioned Throughput pricing, charged by the hour, you have the flexibility to choose between 1-month or 6-month commitment terms.

**Q: Where can I find customer references for Amazon Bedrock?**

Go [here](https://aws.amazon.com/bedrock/testimonials/).

**Q: Where can I find resources for prompt engineering?**

[Prompt Engineering Guide](https://www.promptingguide.ai/).

**Q: Is LangChain mandatory to use Amazon Bedrock?**

No. You can interact with Amazon Bedrock using the [Bedrock API](https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html) or language-specific [AWS SDKs](https://aws.amazon.com/developer/tools/). 

**Q: How do I get started with LangChain?**

Go [here](https://python.langchain.com/docs/get_started/introduction).

**Q: Where can I find pricing information for the AWS services used in this notebook?**

- Amazon Bedrock pricing - go [here](https://aws.amazon.com/bedrock/pricing/).
- AWS Identity and Access Management (IAM) pricing - free.
- Amazon CloudWatch pricing - go [here](https://aws.amazon.com/cloudwatch/pricing/).
- Amazon SageMaker Notebook instance (or) Amazon SageMaker Studio Notebook pricing - go [here](https://aws.amazon.com/sagemaker/pricing/).